In [3]:
import matplotlib.pyplot as plt

import sys
import os
import json

import numpy as np

import torch
import logging
import tqdm
from sklearn.neighbors import NearestNeighbors

from pytorch_pretrained_bert import modeling, tokenization
from glue.model_setup import load_bert
from shared import model_setup as shared_model_setup
from glue.tasks import RteProcessor
from pytorch_pretrained_bert.tokenization import BertTokenizer

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

device = torch.device("cpu")
bert_model_name = "bert-large-uncased"

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
mnli_path = "/Users/thibault/Downloads/mnli.p"
conf_file = "/Users/thibault/.pytorch_pretrained_bert/"
# all_state = shared_model_setup.load_overall_state(mnli_path)

os.environ['BERT_ALL_DIR'] = conf_file
# stilts_mnli_large = load_bert("REGRESSION", bert_model_name, "model_only", all_state, 1)

In [5]:
rte_path = "/Users/thibault/Downloads/mnli__rte.p"
all_state_rte = shared_model_setup.load_overall_state(rte_path)
stilts_rte_large = load_bert("EXTRACTION", bert_model_name, "model_only", all_state_rte, 1)

04/27/2019 17:46:54 - INFO - pytorch_pretrained_bert.modeling -   from_state_dict: Dropping classifier.weight
04/27/2019 17:46:54 - INFO - pytorch_pretrained_bert.modeling -   from_state_dict: Dropping classifier.bias


In [6]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, verbose=True):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_map[example.label]
        if verbose and ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [7]:
rte_processor = RteProcessor()
train_rte = rte_processor.get_train_examples('data/RTE/')
label_list = rte_processor.get_labels()

In [29]:
val_rte = rte_processor.get_dev_examples('data/RTE')

In [8]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)

04/27/2019 17:46:55 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-vocab.txt from cache at /Users/thibault/.pytorch_pretrained_bert/9b3c03a36e83b13d5ba95ac965c9f9074a99e14340c523ab405703179e79fc46.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [9]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id



In [10]:
train_features = convert_examples_to_features(train_rte, label_list, 128, tokenizer)

04/27/2019 17:47:00 - INFO - __main__ -   *** Example ***
04/27/2019 17:47:00 - INFO - __main__ -   guid: train-1
04/27/2019 17:47:00 - INFO - __main__ -   tokens: [CLS] no weapons of mass destruction found in iraq yet . [SEP] weapons of mass destruction found in iraq . [SEP]
04/27/2019 17:47:00 - INFO - __main__ -   input_ids: 101 2053 4255 1997 3742 6215 2179 1999 5712 2664 1012 102 4255 1997 3742 6215 2179 1999 5712 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/27/2019 17:47:00 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/27/2019 17:47:00 - INFO - __main__ -   se

04/27/2019 17:47:00 - INFO - __main__ -   label: not_entailment (id = 1)


In [31]:
dev_features = convert_examples_to_features(val_rte, label_list, 128, tokenizer)

04/27/2019 19:04:12 - INFO - __main__ -   *** Example ***
04/27/2019 19:04:12 - INFO - __main__ -   guid: dev-1
04/27/2019 19:04:12 - INFO - __main__ -   tokens: [CLS] dana reeve , the widow of the actor christopher reeve , has died of lung cancer at age 44 , according to the christopher reeve foundation . [SEP] christopher reeve had an accident . [SEP]
04/27/2019 19:04:12 - INFO - __main__ -   input_ids: 101 11271 20726 1010 1996 7794 1997 1996 3364 5696 20726 1010 2038 2351 1997 11192 4456 2012 2287 4008 1010 2429 2000 1996 5696 20726 3192 1012 102 5696 20726 2018 2019 4926 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/27/2019 19:04:12 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

04/27/2019 19:04:12 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/27/2019 19:04:12 - INFO - __main__ -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
04/27/2019 19:04:12 - INFO - __main__ -   label: entailment (id = 0)


In [34]:
len_rte_val = len(dev_features)
all_emb_val = []
stilts_rte_large.eval()
print(len_rte_val)
k = 32
for i in tqdm.tqdm_notebook(range(0, len_rte_val, k), total=len_rte_val // k):
    feat = dev_features[i:i+k]
    input_ids = torch.tensor([f.input_ids for f in feat], dtype=torch.long)
    input_mask = torch.tensor([f.input_mask for f in feat], dtype=torch.long)
    segment_ids = torch.tensor([f.segment_ids for f in feat], dtype=torch.long)
    _, res = stilts_rte_large(input_ids, segment_ids, input_mask, False)
    all_emb_val.append(res.detach().numpy())
    del _, res

277


In [13]:
len_rte = len(train_features)
all_emb = []
stilts_rte_large.eval()
print(len_rte)
k = 32
for i in tqdm.tqdm_notebook(range(0, len_rte, k), total=len_rte // k):
    feat = train_features[i:i+k]
    input_ids = torch.tensor([f.input_ids for f in feat], dtype=torch.long)
    input_mask = torch.tensor([f.input_mask for f in feat], dtype=torch.long)
    segment_ids = torch.tensor([f.segment_ids for f in feat], dtype=torch.long)
    _, res = stilts_rte_large(input_ids, segment_ids, input_mask, False)
    all_emb.append(res.detach().numpy())
    del _, res

2490


In [14]:
all_emb = np.concatenate(all_emb)

In [74]:
!ls data/

RTE      cloze    winograd wnli


In [36]:
all_emb_val = np.concatenate(all_emb_val)

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [78]:
nn = KNeighborsClassifier(n_neighbors=5)

nn.fit(all_emb, [)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
lr = LogisticRegression()
lr = RandomForestClassifier()


In [83]:
X_train, y_train = all_emb, [f.label == "entailment" for f in train_rte]

X_val, y_val = all_emb_val, [f.label == "entailment" for f in val_rte]

In [89]:
lr.fit(X_train, y_train)
print((lr.predict(X_val) == y_val).mean())

0.8483754512635379


/anaconda3/envs/biobert/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [87]:
preds = nn.predict(all_emb_val)

(preds == ).mean()

SyntaxError: invalid syntax (<ipython-input-87-8adaa6617102>, line 3)

In [41]:
n = 21
print(train_rte[n].text_a)
print(train_rte[n].text_b)

Many hopes are riding on the sale of Talisman's holdings in Palm Beach and Hendry counties, which Vice President Al Gore announced with much fanfare last year at the 50th anniversary of Everglades National Park.
Everglades National Park is located in Florida.


In [42]:
n = 0
print(train_rte[n].text_a)
print(train_rte[n].text_b)

No Weapons of Mass Destruction Found in Iraq Yet.
Weapons of Mass Destruction Found in Iraq.
